In [1]:
import numpy as np
import scipy as sp
import MDAnalysis as mdan
from MDAnalysis.analysis import rdf as mAnRDF
from MDAnalysis.analysis import distances as mAnDist
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()
sns.set_style("white")
sns.set_style("ticks")
sns.set_context("poster")
import pickle

In [2]:
class CG:
    def __init__(self, ag):
        self.ag = ag
        self.universe = self.ag.universe
        self.trajectory = self.ag.universe.trajectory

    @property
    def positions(self):
        return np.array([self.ag.center_of_mass()])

    def __len__(self):
        return 1

In [ ]:
###Calculation of N(r) for Cl- and K+ amid 4 replicates
no_CL,no_OW,no_K=[],[],[]
#     water_hist=[]
from MDAnalysis import transformations as trans

for trajectory in range(4):
    md_uni=mdan.Universe('Chloride/traj_{:d}/npt_production.tpr'.format(trajectory+1),'Chloride/traj_{:d}/npt_production.xtc'.format(trajectory+1))
    
    CL=md_uni.select_atoms('name CL') ###This is changed to 'name OE1 and resname GLU' for the calculation with Glu-
    OW=md_uni.select_atoms('name OW ')
    OW_total=len(md_uni.select_atoms('name OW'))
    CL_total=len(CL)
    print(CL_total)
    K=md_uni.select_atoms('name K ')
    K_total=len(md_uni.select_atoms('name K'))
    protein=md_uni.select_atoms('resnum 1:3')
    not_protein=md_uni.select_atoms('not protein')
    workflow = [trans.unwrap(protein),  # unwrap all fragments
            trans.center_in_box(protein, # move atoms so protein
                                wrap=True), # is centered
            trans.wrap(not_protein)
           ]
    md_uni.trajectory.add_transformations(*workflow)
    
    cg_atom=CG(protein)
    
    
    for ts in md_uni.trajectory[5000:]:

        dist_arr_CL = mAnDist.distance_array(CL.positions, # reference
                                        cg_atom.positions, # configuration
                                        box=md_uni.dimensions)  
        dist_arr_OW = mAnDist.distance_array(OW.positions, # reference
                                        cg_atom.positions, # configuration
                                        box=md_uni.dimensions)
        dist_arr_K = mAnDist.distance_array(K.positions, # reference
                                        cg_atom.positions, # configuration
                                        box=md_uni.dimensions)
        
        R_CL=[np.min(dist_arr_CL[jk]) for jk in range((CL_total))]
        R_OW=[np.min(dist_arr_OW[jk]) for jk in range(len(OW))]
        R_K=[np.min(dist_arr_K[jk]) for jk in range(len(K))]
        
        hist_CL,bins_CL=np.histogram(R_CL,bins=np.arange(0,34,0.2))
        hist_OW,bins_OW=np.histogram(R_OW,bins=np.arange(0,34,0.2))
        hist_K,bins_K=np.histogram(R_K,bins=np.arange(0,34,0.2))
        
        N_CL=[np.sum(hist_CL[:(ele+1)]) for ele in range(len(hist_CL))]
        N_K=[np.sum(hist_K[:(ele+1)]) for ele in range(len(hist_K))]
        N_OW=[np.sum(hist_OW[:(ele+1)]) for ele in range(len(hist_OW))]
        
        no_CL.append(N_CL)
        no_OW.append(N_OW)
        no_K.append(N_K)
    
np.save("no_CL.npy",no_CL)
np.save("no_K.npy",no_K)
np.save("no_OW.npy",no_OW)




In [ ]:
#### Calculation and plotting of Preferential interaction coefficients using N(r)

color_wa=plt.rcParams['axes.prop_cycle'].by_key()['color']
i=0
fd='New_data/'
import csv

plt.figure(figsize=(2.75,2.5), dpi=600)
mean_anion_CL,std_anion_CL=[],[]
for AA in ["D","G","K","Q","R","S"]:
    folder_name="500mM_Chloride/"+AA+"/298K_unbiased/Pref_coeff/"
    
    no_CL=np.load(folder_name+'no_CL.npy')
    no_OW=np.load(folder_name+'no_OW.npy')
    no_K=np.load(folder_name+'no_K.npy')

    mean_CL=np.mean(no_CL,axis=0)
    mean_OW=np.mean(no_OW,axis=0)
    mean_K=np.mean(no_K,axis=0)
    radius=np.arange(0,34,0.2)
    r_mid=np.arange(0.1,33.7,0.2)

    ratio_CL_OW=[(mean_CL[ele+1]-mean_CL[ele])/(mean_OW[ele+1]-mean_OW[ele]) for ele in range(len(no_OW[0])-1)]
    
    bulk_mean_ratio=np.mean(ratio_CL_OW[125:])
    gamma_CL=gamma_bulk_calc(mean_CL,mean_OW,bulk_mean_ratio)
    ### Putting errorbars 
    sigma_sq_N_CL,sigma_sq_N_OW=np.var(no_CL,axis=0),np.var(no_OW,axis=0)
    sigma_sq_ratio=np.var(ratio_CL_OW[125:])
    std_sq=np.sqrt((sigma_sq_N_CL+bulk_mean_ratio*bulk_mean_ratio*sigma_sq_N_OW+mean_OW*mean_OW*sigma_sq_ratio)/(len(no_OW)))
    ###Obtained standard error of the mean
    
    plt.errorbar(radius[1:],gamma_CL,std_sq,color=color_wa[i],label=AA,lw=1,elinewidth=0.01,capsize=0.1)
    
    header=['r(Angstrom)','gamma(r)','Std_error_of_the_mean']
    data=[[radius[ele+1],gamma_CL[ele],std_sq[ele]] for ele in range(len(gamma_CL))]
    
    with open(fd+AA+'_gamma_CL'+'.csv', 'w', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        writer.writerow(header)

        writer.writerows(data)
        f.close()

    mean_anion_CL.append(gamma_CL[125])
    Var_gamma=std_sq[125]
    std_anion_CL.append((Var_gamma))
    
    i+=1
i=0 
mean_anion_GLU,std_anion_GLU=[],[]
for AA in ["D","G","K","Q","R","S"]:
    folder_name="500mM_Glutamate/"+AA+"/298K_unbiased/Pref_coeff/"
    
    no_CL=np.load(folder_name+'no_GLU.npy')
    no_OW=np.load(folder_name+'no_OW.npy')
    no_K=np.load(folder_name+'no_K.npy')

    mean_CL=np.mean(no_CL,axis=0)
    mean_OW=np.mean(no_OW,axis=0)
    mean_K=np.mean(no_K,axis=0)
    radius=np.arange(0,34,0.2)
    r_mid=np.arange(0.1,33.7,0.2)

    ratio_CL_OW=[(mean_CL[ele+1]-mean_CL[ele])/(mean_OW[ele+1]-mean_OW[ele]) for ele in range(len(no_OW[0])-1)]
    ratio_K_OW=[(mean_K[ele+1]-mean_K[ele])/(mean_OW[ele+1]-mean_OW[ele]) for ele in range(len(no_OW[0])-1)]
    bulk_mean_ratio=np.mean(ratio_CL_OW[125:])
    
    gamma_CL=gamma_bulk_calc(mean_CL,mean_OW,bulk_mean_ratio)
    
    ### Putting errorbars
    sigma_sq_N_CL,sigma_sq_N_OW=np.var(no_CL,axis=0),np.var(no_OW,axis=0)
    sigma_sq_ratio=np.var(ratio_CL_OW[125:])
    std_sq=np.sqrt((sigma_sq_N_CL+bulk_mean_ratio*bulk_mean_ratio*sigma_sq_N_OW+mean_OW*mean_OW*sigma_sq_ratio)/(len(no_OW)))
    
    plt.errorbar(radius[1:],gamma_CL,std_sq,color=color_wa[i],lw=1,ls='--',elinewidth=0.01,capsize=0.1)
    
    header=['r(Angstrom)','gamma(r)','Std_error_of_the_mean']
    data=[[radius[ele+1],gamma_CL[ele],std_sq[ele]] for ele in range(len(gamma_CL))]
    
    with open(fd+AA+'_gamma_GLU'+'.csv', 'w', newline='') as f:
        writer = csv.writer(f)

        writer.writerow(header)

        writer.writerows(data)
        f.close()
    
    mean_anion_GLU.append(gamma_CL[125])
    Var_gamma=std_sq[125]
    std_anion_GLU.append((Var_gamma))
    
    i+=1

#plt.xlim([0,25])    
plt.yticks((-1,-0.75,-0.5,-0.25,0.0,0.25,0.5,0.75),fontsize=12)
plt.xticks(fontsize=12)
plt.ylabel(r'$\mathrm{\Gamma_{anion} (r)}$',fontsize=12)
plt.xlabel(r'radial distance, r $\mathrm{(\AA)}$',fontsize=12)
plt.text(0,0.35,r"$\mathrm{\;\;\;- Cl^-}$",fontsize=12)
plt.text(0,0.55,r"$\mathrm{-- Glu^-}$",fontsize=12)
plt.legend(handlelength=1.75,ncols=3,frameon=False,columnspacing=0.75,fontsize=12)
plt.savefig(fd+'new_gamma.pdf',bbox_inches='tight')
plt.show()

In [ ]:
### Python code to calculate Radial distribution functions for amino acids' functional groups and Chloride anions

def _get_RDFs_of(md_univ_obj, atom_sel_1, atom_sel_2, 
                 num_bins=100, x_range=(1,50), 
                 start=0, stop=25000, step=1, **rdf_kwargs):
    """
    Convenient wrapper like function that takes a MDAnalysis universe object, and the two atom selection strings.
    Given the atom selection strings, we calculate the inter-atom RDF.
    """
    dum_atoms_1 = md_univ_obj.select_atoms(atom_sel_1)
    dum_atoms_2 = md_univ_obj.select_atoms(atom_sel_2)
    
    dum_rdf      = mAnRDF.InterRDF(dum_atoms_1, dum_atoms_2, 
                                   nbins=num_bins, range=x_range, 
                                   start=start, stop=stop, step=step,
                                  **rdf_kwargs)
    dum_rdf.run()
    
    the_dum_rdf = dum_rdf.rdf
    the_dum_xar = dum_rdf.bins
    
    return [the_dum_rdf, the_dum_xar]


NZ_CL_rdf,N_CL_rdf,O_CL_rdf,CA_CL_rdf=[],[],[],[]
NZ_OW_rdf,N_OW_rdf,O_OW_rdf,CA_OW_rdf=[],[],[],[]
NZ_K_rdf,N_K_rdf,O_K_rdf,CA_K_rdf=[],[],[],[]

for trajectory in range(4):
    md_uni=mdan.Universe('traj_{:d}/npt_production.gro'.format(trajectory+1),'traj_{:d}/npt_production.xtc'.format(trajectory+1))
    

    N_CL_rdf.append(_get_RDFs_of(md_uni, 'id 7', 'name CL',
                            x_range=(0, 36), num_bins=180,
                            start=5000, stop=25000, step=1))
    NZ_CL_rdf.append(_get_RDFs_of(md_uni, 'id 23', 'name CL',
                            x_range=(0, 36), num_bins=180,
                            start=5000, stop=25000, step=1))
    O_CL_rdf.append(_get_RDFs_of(md_uni, 'id 28', 'name CL',
                            x_range=(0, 36), num_bins=180,
                            start=5000, stop=25000, step=1))
    CA_CL_rdf.append(_get_RDFs_of(md_uni, 'id 9', 'name CL',
                            x_range=(0, 36), num_bins=180,
                            start=5000, stop=25000, step=1))
    
    
    ### OW
    N_OW_rdf.append(_get_RDFs_of(md_uni, 'id 7', 'name OW',
                            x_range=(0, 36), num_bins=180,
                            start=5000, stop=25000, step=1))
    NZ_OW_rdf.append(_get_RDFs_of(md_uni, 'id 23', 'name OW',
                            x_range=(0, 36), num_bins=180,
                            start=5000, stop=25000, step=1))
    O_OW_rdf.append(_get_RDFs_of(md_uni, 'id 28', 'name OW',
                            x_range=(0, 36), num_bins=180,
                            start=5000, stop=25000, step=1))
    CA_OW_rdf.append(_get_RDFs_of(md_uni, 'id 9', 'name OW',
                            x_range=(0, 36), num_bins=180,
                            start=5000, stop=25000, step=1))
    
    ### K+
    N_K_rdf.append(_get_RDFs_of(md_uni, 'id 7', 'name K',
                            x_range=(0, 36), num_bins=180,
                            start=5000, stop=25000, step=1))
    NZ_K_rdf.append(_get_RDFs_of(md_uni, 'id 23', 'name K',
                            x_range=(0, 36), num_bins=180,
                            start=5000, stop=25000, step=1))
    O_K_rdf.append(_get_RDFs_of(md_uni, 'id 28', 'name K',
                            x_range=(0, 36), num_bins=180,
                            start=5000, stop=25000, step=1))
    CA_K_rdf.append(_get_RDFs_of(md_uni, 'id 9', 'name K',
                            x_range=(0, 36), num_bins=180,
                            start=5000, stop=25000, step=1))
    
    
new_dir="RDF_new_bin_0_20"

np.save(new_dir+"/N_CL_rdf.npy",N_CL_rdf)
np.save(new_dir+"/NZ_CL_rdf.npy",NZ_CL_rdf)
np.save(new_dir+"/O_CL_rdf.npy",O_CL_rdf)
np.save(new_dir+"/CA_CL_rdf.npy",CA_CL_rdf)


np.save(new_dir+"/N_OW_rdf.npy",N_OW_rdf)
np.save(new_dir+"/NZ_OW_rdf.npy",NZ_OW_rdf)
np.save(new_dir+"/O_OW_rdf.npy",O_OW_rdf)
np.save(new_dir+"/CA_OW_rdf.npy",CA_OW_rdf)


np.save(new_dir+"/N_K_rdf.npy",N_K_rdf)
np.save(new_dir+"/NZ_K_rdf.npy",NZ_K_rdf)
np.save(new_dir+"/O_K_rdf.npy",O_K_rdf)
np.save(new_dir+"/CA_K_rdf.npy",CA_K_rdf)

